In [28]:
#Las librerías utilizadas en este documento son:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn import metrics
from matplotlib.ticker import ScalarFormatter
from matplotlib import gridspec
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
sns.set()

## En esta sección evaluremos la selección de predictores utilizando regularización ridge y lasso

In [29]:
#leemos el df creado para el barrio de caballito
df = pd.read_csv('./data/data_limpio_gdf_caballito.csv')

Variables predictoras: latitud, longitud y superficie total

Variable objetivo: precio en usd

Volvemos a presentar las metricas obtenidas en la regresión multiple con OLS para las variables mencionadas para comparar con las metricas obtenidas en Lasso y Ridge

In [30]:
X = df[['lat', 'lon', 'sup_m2_total']]
y = df['precio_usd']
# Normalizamos los datos
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# Dividimos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_sm = sm.add_constant(X_train)
X_test_sm = scaler.transform(X_test)

model = sm.OLS(y_train, X_train_sm).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             precio_usd   R-squared:                       0.402
Model:                            OLS   Adj. R-squared:                  0.401
Method:                 Least Squares   F-statistic:                     308.5
Date:                Sun, 04 Dec 2022   Prob (F-statistic):          3.83e-153
Time:                        08:39:31   Log-Likelihood:                -18210.
No. Observations:                1380   AIC:                         3.643e+04
Df Residuals:                    1376   BIC:                         3.645e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.901e+05   3508.946     54.166      0.0

c:\Users\LuisD\anaconda3\envs\dh\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Para un nivel de confianza de 0.05, el p-value del modelo es 0.7007 para la variable latitud por lo que se puede decir que el modelo es peor que lo esperado por el azar para esa variable, el coeficiente parcial de regresión es 0 y no tiene correlación con la varible objetivo.
los p-value de longitud y superficie total (0.043 y 0.000) no están indicando que las variables contribuyen a la preddicción de nuestra variable objetivo. 

Ridge

In [31]:
# Probamos con regularización ridge
lm_ridge = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13))

model_ridge = lm_ridge.fit(X_train, y_train)

print ('Modelo Ridge:')
print('hiperparametro alpha: ', model_ridge.alpha_)
print('coeficientes ajustados: ', model_ridge.coef_)
print('intercepto: ', model_ridge.intercept_)
print('R2 train: ', model_ridge.score(X_train, y_train))
print('R2 test: ', model_ridge.score(X_test, y_test))

Modelo Ridge:
hiperparametro alpha:  10.0
coeficientes ajustados:  [ -1430.05394653   7607.3773849  112611.84153164]
intercepto:  190057.85686628934
R2 train:  0.40209656966130336
R2 test:  0.42910372310449285


Este método consigue minimizar la influencia sobre el modelo de los predictores menos relacionados con la variable respuesta disminuyendolos sin llegar a cero.

Vemos como al igual que el modelo OLS también nos indica que la variable latitud está muy poco correlacionada con el precio usd

Existe mayor influencia de la variable superficie total que longitud al explicar la variable precio usd

El valor promedio de la variable objetivo permanece sin mayor cambio en los modelos OLS y LASSO

El R2 de test en Ridge explica el 43% de la variabilidad observada en la variable precio en usd.

Los R2 de train y test son similares e indican que no hay overfitting y las pruebas en test resultan mejores que en train

Lasso

In [32]:
# Probamos con regularización lasso

lm_lasso = linear_model.LassoCV(alphas=np.logspace(-6, 6, 13), cv=5)

model_lasso = lm_lasso.fit(X_train, y_train)

print ('Modelo Lasso:')
print('hiperparametro alpha : ', model_lasso.alpha_)
print('coeficientes ajustados: ', model_lasso.coef_)
print('intercepto: ', model_lasso.intercept_)
print('R2 train: ', model_lasso.score(X_train, y_train))
print('R2 test: ', model_lasso.score(X_test, y_test))


Modelo Lasso:
hiperparametro alpha :  10.0
coeficientes ajustados:  [ -1397.7563068    7621.77765958 113536.70135065]
intercepto:  190066.84416873095
R2 train:  0.40212354841583564
R2 test:  0.42304244154963166


La regularización con Lasso tiene el efecto de forzar a que los coeficientes de los predictores tiendan a cero; un predictor con coeficiente de regresión cero no influye en el modelo, por lo que consigue excluir los predictores que no son relevantes.

En este caso la regularización con lasso no llevó ninguna variable a cero, sin embargo nos sigue mostrando una mejor explicación de precio usd con superficie total principalmente y longitud.

Los R2 de train y test son similares e indican que no hay overfitting y las pruebas en test resultan mejores que en train

### Analizaremos el impacto de la variables ambientes en la predicción de la variable precio. 

    "ver la explicación al final de la sección"

In [33]:
df_0_ambientes_cat = df[['ambientes_cat', 'precio_usd', 'municipio']]

In [34]:
# Creamos las variables dummies para la variable categórica de ambientes
df_0_ambientes_cat = pd.get_dummies(df_0_ambientes_cat, columns=['ambientes_cat'], drop_first=True)
print(df_0_ambientes_cat.shape)
df_0_ambientes_cat.head()

(1726, 6)


,precio_usd,municipio,ambientes_cat_1,ambientes_cat_2,ambientes_cat_3,ambientes_cat_4 o mas
0,110000.0,Caballito,0,0,1,0
1,170500.0,Caballito,0,1,0,0
2,220410.0,Caballito,0,0,1,0
3,222780.0,Caballito,0,0,1,0
4,108000.0,Caballito,0,0,1,0


In [35]:
df_cat = pd.concat([df_0_ambientes_cat, df["sup_m2_total"]],axis=1, join='inner')
print(df_cat.shape)
df_cat.head(4)

(1726, 7)


,precio_usd,municipio,ambientes_cat_1,ambientes_cat_2,ambientes_cat_3,ambientes_cat_4 o mas,sup_m2_total
0,110000.0,Caballito,0,0,1,0,50.0
1,170500.0,Caballito,0,1,0,0,46.0
2,220410.0,Caballito,0,0,1,0,75.0
3,222780.0,Caballito,0,0,1,0,75.0


In [36]:
# Asignamos las variables predictoras

X_cat = df_cat[['ambientes_cat_1', 'ambientes_cat_2', 'ambientes_cat_3', 'ambientes_cat_4 o mas','sup_m2_total']]
y = df_cat['precio_usd']

# Normalizamos los datos
scaler = StandardScaler()
scaler.fit(X_cat)
X_cat = scaler.transform(X_cat)
# X_cat

# Dividimos en train y test
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat, y, test_size=0.2, random_state=42)
#y_test_cat

# Instanciamos el modelo y lo entrenamos
lr_cat = linear_model.LinearRegression()
lr_cat.fit(X_train_cat, y_train_cat)

LinearRegression()

In [37]:
# Vemos los coeficientes

print('Coeficientes: ', lr_cat.coef_)
print('Intercepto: ', lr_cat.intercept_)

# Calculamos el R2
print('R2: ', r2_score(y_test_cat, lr_cat.predict(X_test_cat)))

Coeficientes:  [-20073.91160918 -10929.91768949  -2374.43192836  20937.89317579
  88809.12663528]
Intercepto:  189782.49935610176
R2:  0.5574478410028677


In [38]:
# Modelamos con statsmodels

X_train_sm_2 = sm.add_constant(X_train_cat)
model_stats = sm.OLS(y_train_cat, X_train_sm_2).fit()

print(model_stats.summary())

                            OLS Regression Results                            
Dep. Variable:             precio_usd   R-squared:                       0.423
Model:                            OLS   Adj. R-squared:                  0.421
Method:                 Least Squares   F-statistic:                     201.9
Date:                Sun, 04 Dec 2022   Prob (F-statistic):          1.78e-161
Time:                        08:39:33   Log-Likelihood:                -18184.
No. Observations:                1380   AIC:                         3.638e+04
Df Residuals:                    1374   BIC:                         3.641e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.898e+05   3449.223     55.022      0.0

In [39]:
# Probamos con regularización ridge

lm_ridge_2 = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13, 20,30))

model_ridge_2 = lm_ridge_2.fit(X_train_cat, y_train_cat)

print(lm_ridge_2.alpha_)
print(lm_ridge_2.coef_)
print(model_ridge_2.score(X_test_cat, y_test_cat))

30.0
[-18546.027514    -9295.463723     -464.62797334  23977.70918281
  85579.30194482]
0.5710652024376902


In [40]:
# Probamos con regularización lasso

lm_lasso_2 = linear_model.LassoCV(alphas=np.logspace(-6, 6, 13,20,30), cv=5)

model_lasso_2 = lm_lasso_2.fit(X_train_cat, y_train_cat)

print(lm_lasso_2.alpha_)
print(lm_lasso_2.coef_)
print(model_lasso_2.score(X_test_cat, y_test_cat))

900.0
[-16374.47209198  -7261.98032139      0.          23376.8604373
  88450.53240727]
0.5613624945248672


Al modelar las categorias ambientes obtenemos:

- OLS: Para un nivel de confianza de 0.05, los p-value del modelo para las variables ambientes (1 a 4+) son mucho mayores al nivel de confianza por lo que se puede inferir que los coeficientes parciales de regresión son iguales a 0 y no tiene correlación con la varible objetivo.

- Ridge:vemos que la influencia sobre el modelo de los predictores ambientes (1 ambiente a 3 ambientes) están muy poco relacionados con la variable respuesta por lo que los vemos disminuidos.

   para mas de 4 ambientes y superficie vemos que existe correlación

- Lasso: vemos que la influencia sobre el modelo de los predictores ambientes (1 ambiente a 2 ambientes) están muy poco relacionados con la variable respuesta por lo que los vemos disminuidos.
   
   No hay influencia de los immuebles de 3 ambientes con la variable precio pues el valor del coeficiente es cero.

   La mayor respuesta para explicar el precio usd de las propiedades es la superficie total